In [ ]:
# === CELL 1: SETUP ===
!pip install -q yfinance

import numpy as np
import pandas as pd
import yfinance as yf
import json
import time
import random
from datetime import datetime
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

print("🧬 Evolution Optimizer Ready")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# === CELL 2: R&D DISCOVERY MODE CONFIG ===
# 🔬 EXPERIMENTAL - Let the AI explore freely and discover new strategies!

EVOLUTION_CONFIG = {
    'population_size': 100,     # BIG population for diversity
    'generations': 60,          # Long run for deep exploration
    'mutation_rate': 0.5,       # HIGH mutation - try wild ideas!
    'crossover_rate': 0.5,      # Balance breeding vs mutation
    'elite_keep': 2,            # Only keep TOP 2 - force exploration
    'tournament_size': 3,       # Small tournaments = more randomness
    'restart_every': 12,        # Inject chaos frequently
    'wild_card_pct': 0.1,       # 10% completely random each gen
}

# === PREVIOUS BEST (starting point to improve on) ===
PREVIOUS_BEST = {
    'rsi_oversold': 21,
    'rsi_overbought': 76,
    'momentum_min': 4,
    'bounce_min': 8,
    'drawdown_trigger': -6,
    'profit_target_1': 14,
    'profit_target_2': 25,
    'stop_loss': -19,
    'trailing_stop': 11,
    'max_hold_days': 32,
    'position_size': 0.21,
    'max_positions': 11,
    # Previous results to beat:
    '_prev_return': 31.3,
    '_prev_win_rate': 71.1,
    '_prev_sharpe': 3.34
}

# === ULTRA-WIDE EXPLORATION RANGES ===
# Let the AI discover things we never thought of!
EVOLVABLE_PARAMS = {
    # === ENTRY - EXPLORE EXTREMES ===
    'rsi_oversold': [10, 60, 21],           # Even try buying at RSI 50-60!
    'rsi_overbought': [50, 95, 76],         # Maybe sell earlier? Later?
    'momentum_min': [0, 30, 4],             # What if we need 0% or 30%?
    'bounce_min': [1, 25, 8],               # Tiny bounces? Huge bounces?
    'drawdown_trigger': [-30, 0, -6],       # Buy at -30% crash? Or 0%?
    
    # === EXIT - WILD EXPERIMENTATION ===
    'profit_target_1': [5, 40, 14],         # Take profits at 5%? 40%?
    'profit_target_2': [10, 80, 25],        # Let it run to 80%?
    'stop_loss': [-35, -5, -19],            # Super tight? Super wide?
    'trailing_stop': [3, 30, 11],           # Trail close? Far?
    'max_hold_days': [5, 120, 32],          # Day trade? Hold 4 months?
    
    # === POSITION SIZING - HIGH IMPACT ===
    'position_size': [0.05, 0.50, 0.21],    # 5% to 50% per position!
    'max_positions': [3, 30, 11],           # Concentrated or diversified?
    
    # === NEW EXPERIMENTAL PARAMS ===
    'vol_spike_mult': [1.0, 4.0, 2.0],      # Volume spike threshold
    'trend_strength_min': [0.0, 1.0, 0.5],  # Trend alignment required
    'ribbon_tight_threshold': [2, 10, 5],   # EMA ribbon compression
    'rsi_divergence_days': [3, 14, 7],      # RSI divergence lookback
    'momentum_accel': [0.0, 2.0, 0.5],      # Momentum acceleration factor
}

WATCHLIST = [
    # HIGH VOLATILITY (quantum, nuclear, AI) - best for swing
    'IONQ', 'RGTI', 'QUBT', 'SMR', 'OKLO', 'LEU',
    # TECH GROWTH
    'NVDA', 'AMD', 'MRVL', 'CRDO', 'MU', 'APLD', 'SERV',
    # MOMENTUM PLAYS  
    'TSLA', 'META', 'GOOGL', 'HOOD', 'SNOW', 'LUNR',
    # RECOVERY/VALUE
    'RIVN', 'LYFT', 'UUUU', 'CCJ',
    # BENCHMARKS
    'SPY', 'QQQ', 'TQQQ'  # Added TQQQ for leverage testing
]

print("🔬 R&D DISCOVERY MODE - PATTERN EVOLUTION")
print("=" * 60)
print(f"   Population: {EVOLUTION_CONFIG['population_size']} (exploring)")
print(f"   Generations: {EVOLUTION_CONFIG['generations']} (deep search)")
print(f"   Mutation Rate: {EVOLUTION_CONFIG['mutation_rate']} (HIGH - wild ideas)")
print(f"   Wild Cards: {EVOLUTION_CONFIG['wild_card_pct']*100}% random each gen")
print(f"   Parameters: {len(EVOLVABLE_PARAMS)} (incl. NEW experimental)")
print(f"   Tickers: {len(WATCHLIST)}")
print(f"\n📊 BASELINE TO BEAT:")
print(f"   Return: +{PREVIOUS_BEST['_prev_return']}%")
print(f"   Win Rate: {PREVIOUS_BEST['_prev_win_rate']}%")
print(f"   Sharpe: {PREVIOUS_BEST['_prev_sharpe']}")
print(f"\n🎯 GOAL: Discover something BETTER or learn why current is optimal")

In [ ]:
# === CELL 3: LOAD DATA ===
print("📥 Loading data...")

data_dict = {}
for ticker in WATCHLIST:
    try:
        df = yf.download(ticker, period='2y', progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df = df.reset_index()
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if len(df) > 100:
            data_dict[ticker] = df
            print(f"   ✓ {ticker}: {len(df)} days")
    except Exception as e:
        print(f"   ✗ {ticker}: {e}")

# Split 70/30 train/test
train_data = {}
test_data = {}
for ticker, df in data_dict.items():
    split = int(len(df) * 0.7)
    train_data[ticker] = df.iloc[:split].reset_index(drop=True)
    test_data[ticker] = df.iloc[split:].reset_index(drop=True)

print(f"\n✅ Train: {len(train_data[list(train_data.keys())[0]])} days")
print(f"✅ Test: {len(test_data[list(test_data.keys())[0]])} days")

In [ ]:
# === CELL 4: ENHANCED FEATURE ENGINE (with experimental features) ===
def compute_features(df):
    df = df.copy()
    c = df['Close'].astype(float)
    h = df['High'].astype(float)
    l = df['Low'].astype(float)
    v = df['Volume'].astype(float)
    
    # === STANDARD FEATURES ===
    for p in [1, 5, 10, 21]:
        df[f'ret_{p}d'] = c.pct_change(p) * 100
    
    # EMAs
    for p in [8, 13, 21, 34, 55]:
        df[f'ema_{p}'] = c.ewm(span=p).mean()
    df['ema_8_rising'] = (df['ema_8'] > df['ema_8'].shift(3)).astype(float)
    
    # Ribbon
    df['ribbon_bullish'] = ((df['ema_8'] > df['ema_13']) & (df['ema_13'] > df['ema_21'])).astype(float)
    df['ribbon_range'] = (df[['ema_8','ema_13','ema_21','ema_34','ema_55']].max(axis=1) - 
                          df[['ema_8','ema_13','ema_21','ema_34','ema_55']].min(axis=1)) / c * 100
    df['ribbon_tight'] = (df['ribbon_range'] < 5).astype(float)
    
    # RSI
    delta = c.diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    df['rsi'] = 100 - (100 / (1 + gain / (loss + 1e-10)))
    
    # MACD
    df['macd'] = c.ewm(span=12).mean() - c.ewm(span=26).mean()
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_rising'] = (df['macd'] - df['macd_signal'] > (df['macd'] - df['macd_signal']).shift(1)).astype(float)
    
    # Volume
    df['vol_ratio'] = v / (v.rolling(20).mean() + 1)
    df['vol_spike'] = (df['vol_ratio'] > 2).astype(float)
    
    # Momentum
    df['mom_5d'] = c.pct_change(5) * 100
    
    # Bounce
    df['low_5d'] = l.rolling(5).min()
    df['bounce'] = (c / (df['low_5d'] + 1e-10) - 1) * 100
    df['bounce_signal'] = ((df['bounce'] > 3) & (df['ema_8_rising'] > 0)).astype(float)
    
    # Trend
    df['trend_align'] = (np.sign(df['ret_5d']) + np.sign(df['ret_10d']) + np.sign(df['ret_21d'])) / 3
    
    # === NEW EXPERIMENTAL FEATURES ===
    
    # RSI Divergence (price makes new low but RSI doesn't)
    df['price_low_5d'] = c.rolling(5).min()
    df['rsi_at_price_low'] = df['rsi'].rolling(5).min()
    df['rsi_divergence'] = ((c <= df['price_low_5d'] * 1.02) & 
                            (df['rsi'] > df['rsi_at_price_low'] + 5)).astype(float)
    
    # Momentum Acceleration (momentum speeding up)
    df['mom_accel'] = df['mom_5d'] - df['mom_5d'].shift(3)
    
    # Volatility Squeeze (low volatility = breakout coming)
    df['atr'] = (h - l).rolling(14).mean()
    df['atr_pct'] = df['atr'] / c * 100
    df['vol_squeeze'] = (df['atr_pct'] < df['atr_pct'].rolling(50).mean() * 0.7).astype(float)
    
    # Price vs Moving Averages
    df['above_ema_21'] = (c > df['ema_21']).astype(float)
    df['above_ema_55'] = (c > df['ema_55']).astype(float)
    df['ema_stack_bullish'] = (df['above_ema_21'] + df['above_ema_55']) / 2
    
    # Relative Strength vs SPY (if we have it)
    df['rel_strength'] = df['ret_21d']  # Will be compared later
    
    # Gap detection
    df['gap_up'] = ((df['Open'] / c.shift(1) - 1) * 100 > 2).astype(float)
    df['gap_down'] = ((df['Open'] / c.shift(1) - 1) * 100 < -2).astype(float)
    
    # Consolidation (price range shrinking)
    df['range_10d'] = (h.rolling(10).max() - l.rolling(10).min()) / c * 100
    df['consolidating'] = (df['range_10d'] < df['range_10d'].rolling(30).mean() * 0.6).astype(float)
    
    # Higher Lows / Higher Highs
    df['higher_low'] = (l > l.shift(5)).astype(float)
    df['higher_high'] = (h > h.shift(5)).astype(float)
    df['uptrend_structure'] = (df['higher_low'] + df['higher_high']) / 2
    
    df = df.replace([np.inf, -np.inf], np.nan).ffill().bfill().fillna(0)
    return df

print("🧠 Computing ENHANCED features...")
train_features = {t: compute_features(df) for t, df in train_data.items()}
test_features = {t: compute_features(df) for t, df in test_data.items()}
print(f"✅ Features ready - {len(train_features[list(train_features.keys())[0]].columns)} total features")

In [ ]:
# === CELL 5: DNA CLASS (Strategy Genome) ===

class StrategyDNA:
    """A trading strategy's genetic code"""
    
    def __init__(self, params=None):
        if params:
            self.params = params
        else:
            # Random initialization
            self.params = {}
            for name, (min_v, max_v, _) in EVOLVABLE_PARAMS.items():
                if isinstance(min_v, int):
                    self.params[name] = random.randint(min_v, max_v)
                else:
                    self.params[name] = random.uniform(min_v, max_v)
        
        self.fitness = 0
        self.test_fitness = 0
    
    def mutate(self):
        """Randomly mutate one parameter"""
        param = random.choice(list(self.params.keys()))
        min_v, max_v, _ = EVOLVABLE_PARAMS[param]
        
        # Mutation: adjust by 10-30%
        current = self.params[param]
        range_size = max_v - min_v
        mutation = random.uniform(-0.3, 0.3) * range_size
        new_val = current + mutation
        
        # Clamp to bounds
        new_val = max(min_v, min(max_v, new_val))
        
        if isinstance(min_v, int):
            new_val = int(new_val)
        
        self.params[param] = new_val
    
    @staticmethod
    def crossover(parent1, parent2):
        """Breed two strategies"""
        child_params = {}
        for param in parent1.params:
            # 50% chance from each parent, with some blending
            if random.random() < 0.5:
                child_params[param] = parent1.params[param]
            else:
                child_params[param] = parent2.params[param]
            
            # 20% chance to blend
            if random.random() < 0.2:
                blend = (parent1.params[param] + parent2.params[param]) / 2
                min_v, max_v, _ = EVOLVABLE_PARAMS[param]
                if isinstance(min_v, int):
                    blend = int(blend)
                child_params[param] = blend
        
        return StrategyDNA(child_params)
    
    def __repr__(self):
        return f"DNA(fitness={self.fitness:.1f})"

print("✅ DNA class ready")
# Test
test_dna = StrategyDNA()
print(f"   Sample DNA: {test_dna.params}")

In [ ]:
# === CELL 6: EXPERIMENTAL FITNESS EVALUATOR ===

def evaluate_fitness(dna, features_dict, data_dict):
    """
    Run backtest with EXPERIMENTAL entry/exit logic.
    Uses evolved parameters + new experimental features.
    """
    p = dna.params
    balance = 100000
    positions = {}
    trades = []
    history = [balance]
    
    tickers = list(features_dict.keys())
    min_len = min(len(df) for df in data_dict.values())
    
    for day in range(60, min_len - 1):
        prices = {}
        
        # Update positions
        for t in list(positions.keys()):
            if t in data_dict and day < len(data_dict[t]):
                price = float(data_dict[t]['Close'].iloc[day])
                prices[t] = price
                positions[t]['days'] += 1
                if price > positions[t]['max']:
                    positions[t]['max'] = price
        
        for ticker in tickers:
            if day >= len(features_dict[ticker]):
                continue
            
            df = features_dict[ticker]
            price = float(data_dict[ticker]['Close'].iloc[day])
            prices[ticker] = price
            
            # Standard features
            rsi = float(df['rsi'].iloc[day])
            mom = float(df['mom_5d'].iloc[day])
            ret_21d = float(df['ret_21d'].iloc[day])
            ribbon_bull = float(df['ribbon_bullish'].iloc[day])
            macd_rising = float(df['macd_rising'].iloc[day])
            bounce = float(df['bounce'].iloc[day])
            bounce_sig = float(df['bounce_signal'].iloc[day])
            trend = float(df['trend_align'].iloc[day])
            vol_ratio = float(df['vol_ratio'].iloc[day])
            
            # NEW experimental features
            rsi_div = float(df['rsi_divergence'].iloc[day])
            mom_accel = float(df['mom_accel'].iloc[day])
            vol_squeeze = float(df['vol_squeeze'].iloc[day])
            ema_stack = float(df['ema_stack_bullish'].iloc[day])
            consolidating = float(df['consolidating'].iloc[day])
            uptrend = float(df['uptrend_structure'].iloc[day])
            ribbon_range = float(df['ribbon_range'].iloc[day])
            
            # Get evolved experimental params (with defaults)
            vol_spike_mult = p.get('vol_spike_mult', 2.0)
            trend_min = p.get('trend_strength_min', 0.5)
            ribbon_tight_thresh = p.get('ribbon_tight_threshold', 5)
            mom_accel_factor = p.get('momentum_accel', 0.5)
            
            # === SELL LOGIC (with experimental exits) ===
            if ticker in positions:
                pos = positions[ticker]
                pnl = (price / pos['entry'] - 1) * 100
                from_max = (price / pos['max'] - 1) * 100
                
                sell = False
                sell_reason = None
                
                # Standard exits
                if pnl >= p['profit_target_2']:
                    sell, sell_reason = True, 'profit_target_2'
                elif pnl >= p['profit_target_1'] and rsi > p['rsi_overbought']:
                    sell, sell_reason = True, 'profit_target_1_overbought'
                elif pnl <= p['stop_loss']:
                    sell, sell_reason = True, 'stop_loss'
                elif pnl > 15 and from_max < -p['trailing_stop']:
                    sell, sell_reason = True, 'trailing_stop'
                elif pos['days'] > p['max_hold_days'] and pnl > 0:
                    sell, sell_reason = True, 'max_hold_profit'
                
                # EXPERIMENTAL exits
                elif pnl > p['profit_target_1'] and mom_accel < -mom_accel_factor:
                    sell, sell_reason = True, 'momentum_dying'  # Momentum fading
                elif pnl > 10 and ribbon_bull < 1 and ema_stack < 0.5:
                    sell, sell_reason = True, 'trend_breakdown'  # Structure breaking
                
                if sell:
                    balance += pos['shares'] * price
                    trades.append({'pnl': pnl, 'reason': sell_reason, 'ticker': ticker})
                    del positions[ticker]
            
            # === BUY LOGIC (with experimental entries) ===
            else:
                if len(positions) >= int(p['max_positions']):
                    continue
                
                pv = balance + sum(positions[t]['shares'] * prices.get(t, 0) for t in positions)
                if balance / pv < 0.1:
                    continue
                
                buy = False
                buy_reason = None
                signal_strength = 0
                
                # === EVOLVED ENTRY CONDITIONS ===
                
                # 1. Classic Dip Buy (evolved thresholds)
                if rsi < p['rsi_oversold'] and mom < -3:
                    buy, buy_reason = True, 'dip_buy'
                    signal_strength = 1
                
                # 2. Bounce Play
                if bounce > p['bounce_min'] and macd_rising > 0:
                    buy, buy_reason = True, 'bounce'
                    signal_strength = 1
                
                # 3. Nuclear Dip (big drawdown recovery)
                if ret_21d < p['drawdown_trigger'] and macd_rising > 0:
                    buy, buy_reason = True, 'nuclear_dip'
                    signal_strength = 1.5
                
                # 4. Momentum Continuation
                if mom > p['momentum_min'] and macd_rising > 0 and bounce_sig > 0:
                    buy, buy_reason = True, 'momentum_cont'
                    signal_strength = 1
                
                # 5. Trend Continuation
                if trend > trend_min and ribbon_bull > 0 and p['rsi_oversold'] < rsi < p['rsi_overbought']:
                    buy, buy_reason = True, 'trend_cont'
                    signal_strength = 1
                
                # === NEW EXPERIMENTAL ENTRIES ===
                
                # 6. RSI Divergence (price low but RSI higher = reversal)
                if rsi_div > 0 and macd_rising > 0:
                    buy, buy_reason = True, 'rsi_divergence'
                    signal_strength = 1.5
                
                # 7. Volatility Squeeze Breakout
                if vol_squeeze > 0 and mom > 0 and vol_ratio > vol_spike_mult:
                    buy, buy_reason = True, 'vol_squeeze_breakout'
                    signal_strength = 1.3
                
                # 8. Consolidation Breakout
                if consolidating > 0 and mom > p['momentum_min'] * 0.5 and ribbon_bull > 0:
                    buy, buy_reason = True, 'consolidation_breakout'
                    signal_strength = 1.2
                
                # 9. Uptrend Structure + Pullback
                if uptrend > 0.5 and rsi < 45 and ema_stack > 0.5:
                    buy, buy_reason = True, 'uptrend_pullback'
                    signal_strength = 1.3
                
                # 10. EMA Ribbon Compression (about to expand)
                if ribbon_range < ribbon_tight_thresh and macd_rising > 0 and trend > 0:
                    buy, buy_reason = True, 'ribbon_compression'
                    signal_strength = 1.1
                
                if buy:
                    # Scale position by signal strength
                    pos_size = p['position_size'] * signal_strength
                    pos_size = min(pos_size, 0.5)  # Cap at 50%
                    shares = int(balance * pos_size / price)
                    if shares > 0:
                        balance -= shares * price
                        positions[ticker] = {
                            'shares': shares,
                            'entry': price,
                            'max': price,
                            'days': 0,
                            'reason': buy_reason
                        }
        
        pv = balance + sum(positions[t]['shares'] * prices.get(t, 0) for t in positions)
        history.append(pv)
    
    # Liquidate
    for ticker, pos in positions.items():
        if ticker in data_dict:
            price = float(data_dict[ticker]['Close'].iloc[-1])
            balance += pos['shares'] * price
            trades.append({'pnl': (price / pos['entry'] - 1) * 100, 'reason': 'liquidate', 'ticker': ticker})
    
    # === CALCULATE METRICS ===
    total_return = (balance / 100000 - 1) * 100
    winners = len([t for t in trades if t['pnl'] > 0])
    win_rate = winners / max(len(trades), 1)
    num_trades = len(trades)
    
    # Sharpe
    returns = np.diff(history) / (np.array(history[:-1]) + 1e-10)
    sharpe = np.mean(returns) / (np.std(returns) + 1e-10) * np.sqrt(252)
    
    # Max drawdown
    peak = np.maximum.accumulate(history)
    dd = (np.array(history) - peak) / (peak + 1e-10)
    max_dd = np.min(dd)
    
    # Profit Factor
    gross_profit = sum(t['pnl'] for t in trades if t['pnl'] > 0)
    gross_loss = abs(sum(t['pnl'] for t in trades if t['pnl'] < 0))
    profit_factor = gross_profit / max(gross_loss, 1)
    
    # === FITNESS FORMULA (multi-objective) ===
    # Balance return, risk, consistency
    fitness = (
        total_return * 1.0 +                    # Raw return
        (win_rate * 100) * 0.5 +                # Win rate bonus
        sharpe * 20 +                           # Risk-adjusted return
        profit_factor * 10 +                    # Quality of wins
        min(num_trades, 100) * 0.1 -            # Reward activity (up to 100)
        abs(max_dd) * 50                        # Drawdown penalty
    )
    
    return fitness, total_return, win_rate * 100, sharpe, max_dd * 100

print("✅ EXPERIMENTAL Fitness evaluator ready")
print("   - 10 entry signals (5 classic + 5 experimental)")
print("   - Signal strength scaling")
print("   - Multi-objective fitness")

In [ ]:
# === CELL 7: EVOLUTION ENGINE ===

def evolve_population(population, features, data):
    """One generation of evolution"""
    
    # Evaluate all
    for dna in population:
        dna.fitness, _, _, _, _ = evaluate_fitness(dna, features, data)
    
    # Sort by fitness
    population.sort(key=lambda x: x.fitness, reverse=True)
    
    # Keep elite
    new_pop = population[:EVOLUTION_CONFIG['elite_keep']]
    
    # Fill rest with offspring
    while len(new_pop) < EVOLUTION_CONFIG['population_size']:
        # Tournament selection
        def tournament():
            contestants = random.sample(population[:20], EVOLUTION_CONFIG['tournament_size'])
            return max(contestants, key=lambda x: x.fitness)
        
        parent1 = tournament()
        parent2 = tournament()
        
        # Crossover
        if random.random() < EVOLUTION_CONFIG['crossover_rate']:
            child = StrategyDNA.crossover(parent1, parent2)
        else:
            child = StrategyDNA(deepcopy(parent1.params))
        
        # Mutation
        if random.random() < EVOLUTION_CONFIG['mutation_rate']:
            child.mutate()
        
        new_pop.append(child)
    
    return new_pop

print("✅ Evolution engine ready")

In [ ]:
# === CELL 8: R&D EVOLUTION RUN ===
print("=" * 70)
print("🔬 R&D DISCOVERY MODE - PATTERN EVOLUTION")
print("=" * 70)

# Initialize population with diversity
population = [StrategyDNA() for _ in range(EVOLUTION_CONFIG['population_size'])]

# Seed with previous best
population[0] = StrategyDNA(PREVIOUS_BEST.copy())

# Seed with human baseline
human_params = {name: best for name, (_, _, best) in EVOLVABLE_PARAMS.items()}
population[1] = StrategyDNA(human_params)

# Seed some extreme variants for exploration
extreme_aggressive = human_params.copy()
extreme_aggressive['position_size'] = 0.40
extreme_aggressive['stop_loss'] = -30
extreme_aggressive['profit_target_2'] = 60
population[2] = StrategyDNA(extreme_aggressive)

extreme_conservative = human_params.copy()
extreme_conservative['position_size'] = 0.08
extreme_conservative['stop_loss'] = -8
extreme_conservative['profit_target_1'] = 8
population[3] = StrategyDNA(extreme_conservative)

extreme_fast = human_params.copy()
extreme_fast['max_hold_days'] = 7
extreme_fast['profit_target_1'] = 5
extreme_fast['profit_target_2'] = 12
population[4] = StrategyDNA(extreme_fast)

best_ever = None
history = []
discoveries = []  # Track interesting findings

start_time = time.time()

print(f"\n🚀 Starting with {EVOLUTION_CONFIG['population_size']} strategies")
print(f"   Including: Previous Best, Human, Aggressive, Conservative, Fast\n")

for gen in range(EVOLUTION_CONFIG['generations']):
    # Evolve on TRAINING data
    population = evolve_population(population, train_features, train_data)
    
    best = population[0]
    avg_fitness = np.mean([d.fitness for d in population[:10]])
    diversity = np.std([d.fitness for d in population])
    
    # Track best ever
    if best_ever is None or best.fitness > best_ever.fitness:
        best_ever = StrategyDNA(deepcopy(best.params))
        best_ever.fitness = best.fitness
        
        # Log discovery
        discoveries.append({
            'gen': gen,
            'fitness': best.fitness,
            'params': best.params.copy()
        })
        marker = "🆕 NEW BEST!"
    else:
        marker = ""
    
    history.append({
        'gen': gen,
        'best_fitness': best.fitness,
        'avg_fitness': avg_fitness,
        'diversity': diversity
    })
    
    # CHAOS INJECTION: Fresh random DNA
    restart_every = EVOLUTION_CONFIG.get('restart_every', 12)
    if gen > 0 and gen % restart_every == 0:
        num_replace = int(EVOLUTION_CONFIG['population_size'] * 0.25)
        for i in range(-num_replace, 0):
            population[i] = StrategyDNA()
        print(f"   🔄 Gen {gen}: Injected {num_replace} wild cards")
    
    # WILD CARD: Some completely random each generation
    wild_pct = EVOLUTION_CONFIG.get('wild_card_pct', 0.1)
    num_wild = max(1, int(EVOLUTION_CONFIG['population_size'] * wild_pct))
    for i in range(num_wild):
        idx = random.randint(EVOLUTION_CONFIG['elite_keep'], len(population) - 1)
        population[idx] = StrategyDNA()
    
    if gen % 5 == 0:
        elapsed = time.time() - start_time
        print(f"Gen {gen:3d} | Best: {best.fitness:8.1f} | Avg: {avg_fitness:7.1f} | Div: {diversity:6.1f} | {elapsed:.0f}s {marker}")

print("-" * 70)
print(f"\n🏆 R&D COMPLETE in {time.time()-start_time:.0f}s")
print(f"   Best Fitness: {best_ever.fitness:.1f}")
print(f"   Discoveries made: {len(discoveries)}")

In [ ]:
# === CELL 9: TEST BEST ON UNSEEN DATA ===
print("=" * 70)
print("🧪 TESTING EVOLVED STRATEGY ON UNSEEN DATA")
print("=" * 70)

# Test the evolved best
evolved_fitness, evolved_ret, evolved_wr, evolved_sharpe, evolved_dd = evaluate_fitness(
    best_ever, test_features, test_data
)

# Test human baseline
human_dna = StrategyDNA(human_params)
human_fitness, human_ret, human_wr, human_sharpe, human_dd = evaluate_fitness(
    human_dna, test_features, test_data
)

print(f"\n{'Metric':<20} {'Human':>15} {'Evolved':>15} {'Winner':>10}")
print("-" * 60)
print(f"{'Return':<20} {human_ret:>+14.1f}% {evolved_ret:>+14.1f}% {'🧬' if evolved_ret > human_ret else '👤'}")
print(f"{'Win Rate':<20} {human_wr:>14.0f}% {evolved_wr:>14.0f}% {'🧬' if evolved_wr > human_wr else '👤'}")
print(f"{'Sharpe Ratio':<20} {human_sharpe:>15.2f} {evolved_sharpe:>15.2f} {'🧬' if evolved_sharpe > human_sharpe else '👤'}")
print(f"{'Max Drawdown':<20} {human_dd:>14.1f}% {evolved_dd:>14.1f}% {'🧬' if evolved_dd > human_dd else '👤'}")
print(f"{'Fitness':<20} {human_fitness:>15.1f} {evolved_fitness:>15.1f} {'🧬' if evolved_fitness > human_fitness else '👤'}")

improvement = (evolved_ret - human_ret) / max(abs(human_ret), 1) * 100
print(f"\n📈 Evolution improved returns by {improvement:+.1f}%")

In [ ]:
# === CELL 10: EVOLVED PARAMETERS ===
print("=" * 70)
print("🧬 EVOLVED OPTIMAL PARAMETERS")
print("=" * 70)

print(f"\n{'Parameter':<25} {'Human':>12} {'Evolved':>12} {'Change':>12}")
print("-" * 65)

for param in EVOLVABLE_PARAMS:
    human_val = human_params[param]
    evolved_val = best_ever.params[param]
    
    if isinstance(human_val, float):
        change = (evolved_val - human_val) / max(abs(human_val), 0.01) * 100
        print(f"{param:<25} {human_val:>12.2f} {evolved_val:>12.2f} {change:>+11.1f}%")
    else:
        change = evolved_val - human_val
        print(f"{param:<25} {human_val:>12} {evolved_val:>12} {change:>+12}")

print("\n" + "=" * 70)
print("📋 COPY THESE EVOLVED SETTINGS:")
print("=" * 70)
print(json.dumps(best_ever.params, indent=2))

In [ ]:
# === CELL 11: SAVE RESULTS ===
results = {
    'generated_at': datetime.now().isoformat(),
    'evolution_config': EVOLUTION_CONFIG,
    'generations_run': EVOLUTION_CONFIG['generations'],
    'human_baseline': {
        'params': human_params,
        'test_return': human_ret,
        'test_win_rate': human_wr,
        'test_sharpe': human_sharpe,
        'test_max_dd': human_dd
    },
    'evolved_best': {
        'params': best_ever.params,
        'test_return': evolved_ret,
        'test_win_rate': evolved_wr,
        'test_sharpe': evolved_sharpe,
        'test_max_dd': evolved_dd,
        'fitness': best_ever.fitness
    },
    'improvement': {
        'return_pct': evolved_ret - human_ret,
        'win_rate_pct': evolved_wr - human_wr,
        'sharpe_diff': evolved_sharpe - human_sharpe
    },
    'evolution_history': history
}

with open('evolution_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print("✅ Results saved to evolution_results.json")

try:
    from google.colab import files
    files.download('evolution_results.json')
    print("📥 Download started!")
except:
    print("(Not in Colab - file saved locally)")

In [ ]:
# === CELL 12: RECOMMENDATIONS ===
print("=" * 70)
print("📋 FINAL RECOMMENDATIONS")
print("=" * 70)

if evolved_ret > human_ret * 1.1:  # 10% better
    print("\n🧬 USE EVOLVED PARAMETERS!")
    print("   The AI found significantly better settings.")
    rec = best_ever.params
else:
    print("\n👤 STICK WITH HUMAN PARAMETERS (or blend)")
    print("   Evolution didn't find major improvements.")
    rec = human_params

print("\n🎯 RECOMMENDED SETTINGS FOR YOUR TRADING:")
print("-" * 50)
print(f"   RSI Oversold: < {rec['rsi_oversold']:.0f}")
print(f"   RSI Overbought: > {rec['rsi_overbought']:.0f}")
print(f"   Momentum Entry: > {rec['momentum_min']:.0f}%")
print(f"   Bounce Entry: > {rec['bounce_min']:.1f}%")
print(f"   Dip Trigger (21d): < {rec['drawdown_trigger']:.0f}%")
print(f"   Profit Target 1: {rec['profit_target_1']:.0f}%")
print(f"   Profit Target 2: {rec['profit_target_2']:.0f}%")
print(f"   Stop Loss: {rec['stop_loss']:.0f}%")
print(f"   Trailing Stop: {rec['trailing_stop']:.0f}%")
print(f"   Max Hold Days: {rec['max_hold_days']:.0f}")
print(f"   Position Size: {rec['position_size']*100:.0f}%")
print(f"   Max Positions: {int(rec['max_positions'])}")